In [12]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [13]:
n_collect = 20
coverage0 = 0.2

In [14]:
np.concatenate([np.linspace(0, 0.5, 6), np.linspace(0.5, 1, 11)[1:], np.linspace(1, 2, 11)[1:]])

array([0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75,
       0.8 , 0.85, 0.9 , 0.95, 1.  , 1.1 , 1.2 , 1.3 , 1.4 , 1.5 , 1.6 ,
       1.7 , 1.8 , 1.9 , 2.  ])

In [15]:
def collect_vul(pars_post, n_collect=20, coverages=np.linspace(0, 0.8, 33), tp="VulACF"):    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={tp: {'Coverage': k}}, t_end=2031)
            
            idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Plain') or c.startswith('ACF_Vul')]

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            
            mss_stats.append(ms_stats)
            
        
            
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats


def collect_plain(pars_post, n_collect=20, coverages=np.linspace(0, 0.8, 33), cxr=True):    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()

    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={'PlainACF': {'Coverage': k, 'CXR': cxr}}, t_end=2031)
            
            idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Plain')]

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            
            mss_stats.append(ms_stats)
            
    tp = 'PlainACF' if cxr else 'PlainACF_NoCXR'
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats

In [16]:
n_collect = 150
cvs = np.concatenate([np.linspace(0, 0.5, 6), np.linspace(0.5, 1, 11)[1:], np.linspace(1, 2, 11)[1:]])

for folder in ['dy_hi', 'dy_lo']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)

    mss0, mss_stats0 = collect_vul(pars_post, n_collect=n_collect, tp='VulACF', coverages=cvs)
    mss1, mss_stats1 = collect_vul(pars_post, n_collect=n_collect, tp='PlainACF', coverages=cvs)
    mss2, mss_stats2 = collect_plain(pars_post, n_collect=n_collect, cxr=False, coverages=cvs)
    mss = pd.concat([mss0, mss1, mss2])
    mss_stats = pd.concat([mss_stats0, mss_stats1, mss_stats2])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_hi
dy_lo


In [5]:
def collect_fu(pars_post, n_collect=20, coverage=coverage0, tp="VulACF"):
    intvs = {
        'Baseline': {},
        'Vul_0_0': {'VulACF': {'Coverage': coverage, 'FollowUp': 0, 'Duration': 0}},
        'Vul_2_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 2}},
        'Vul_2_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 2}},
        'Vul_3_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 3}},
        'Vul_3_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 3}},
    }
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
        
        for key, intv in intvs.items():
            _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)
            
            idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Vulfu') or c.startswith('ACF_Vul')]

            ms = ms[idx].reset_index().assign(Key = i, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [6]:
for folder in ['dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fu(pars_post, n_collect=n_collect, coverage=0.2)
    mss_stats = mss_stats.fillna(0)
    mss.to_csv(f'{out_path}/Sim_VulACF_followup.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_followup_stats.csv')

dy_hi


In [7]:
mss_stats

,Key,Scenario,Pop0,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,ACF_Vul_Sym,ACF_Vul_Vul,...,ACF_VulFu_Sym,ACF_VulFu_Vul,ACF_VulFu_CXR,ACF_VulFu_Xpert,ACF_VulFu_Yield,ACF_VulFu_TP,ACF_VulFu_DS_Fl,ACF_VulFu_DR_Fl,ACF_VulFu_DR_Sl,Coverage
0,3,Baseline,2.383012,0.113563,0.016300,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
1,3,Vul_0_0,2.383012,0.103960,0.014221,4.413870,4.407599,0.338619,4.407599,4.407599,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
2,3,Vul_2_3,2.383012,0.101413,0.013716,4.413971,4.407599,0.338167,4.407599,4.407599,...,3.497630,0.0,0.0,0.127722,0.005842,0.002407,0.002302,0.0,0.000105,0.2
3,3,Vul_2_6,2.383012,0.101958,0.013830,4.413914,4.407599,0.338275,4.407599,4.407599,...,1.749750,0.0,0.0,0.064311,0.003314,0.001596,0.001526,0.0,0.000070,0.2
4,3,Vul_3_3,2.383012,0.100978,0.013624,4.413998,4.407599,0.338082,4.407599,4.407599,...,4.285012,0.0,0.0,0.156584,0.007321,0.003114,0.002979,0.0,0.000136,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,1493,Vul_0_0,2.386118,0.082931,0.009939,4.418542,4.413343,0.360107,4.413343,4.413343,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
596,1493,Vul_2_3,2.386118,0.081421,0.009688,4.418616,4.413343,0.359889,4.413343,4.413343,...,3.460454,0.0,0.0,0.125937,0.004999,0.001784,0.001705,0.0,0.000079,0.2
597,1493,Vul_2_6,2.386118,0.081911,0.009771,4.418575,4.413343,0.359965,4.413343,4.413343,...,1.730324,0.0,0.0,0.063262,0.002777,0.001169,0.001118,0.0,0.000052,0.2
598,1493,Vul_3_3,2.386118,0.081049,0.009624,4.418634,4.413343,0.359833,4.413343,4.413343,...,4.234880,0.0,0.0,0.154195,0.006230,0.002296,0.002196,0.0,0.000101,0.2


In [8]:
def collect_fux(pars_post, n_collect=20, coverage=coverage0, tp="VulACF"):    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()

    # Test run
    pars = pars_post[0]
    y1, pars = m.find_baseline(pars, 2022)
    _, ms, _ = m.simulate_onward(y1, pars, intv={'VulACF': {'Coverage': 0.01, 'FollowUp': 1, 'Duration': 1}}, t_end=2025)

    idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Vul')]
    print(idx)
    
    #
    
    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
    
        _, ms, _ = m.simulate_onward(y1, pars, intv={}, t_end=2031)

        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        ms_stats = {'Key': i, 'Scenario': 'Baseline', 'Pop0': ms.Pop.iloc[0], 
                    'FollowUp': 0, 'Duration': 0}
        for key in idx[1:]:
            try:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            except KeyError:
                ms_stats[key] = 0

        mss_stats.append(ms_stats)
    
        for n_fu in list(range(7)): # + [24, 30, 36]:
            if n_fu > 0:
                fu = 1 / n_fu
            else:
                fu = 0
            for dur in np.linspace(0, 4, 9):
                intv = {'VulACF': {'Coverage': coverage, 'FollowUp': fu, 'Duration': dur}}
                
                _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)
                try:
                    ms = ms.reset_index().assign(Key = i, FollowUp=fu, Duration=dur)
                    mss.append(ms)

                    ms_stats = {'Key': i, 'Scenario': 'Intervention', 'Pop0': ms.Pop.iloc[0], 
                                'FollowUp': fu, 'Duration': dur}

                    for key in idx[1:]:
                        try:
                            ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
                        except KeyError:
                            ms_stats[key] = 0

                    mss_stats.append(ms_stats)
                except TypeError:
                    pass
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [9]:
for folder in ['dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fux(pars_post, n_collect=n_collect, coverage=0.2)
    mss_stats = mss_stats.fillna(0)
    mss.to_csv(f'{out_path}/Sim_VulACF_fudur.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_fudur_stats.csv')

dy_hi
['Pop', 'IncR', 'MorR', 'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Sym', 'ACF_Vul_Vul', 'ACF_Vul_CXR', 'ACF_Vul_Xpert', 'ACF_Vul_Yield', 'ACF_Vul_TP', 'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 'ACF_VulFu_Footfall', 'ACF_VulFu_Screened', 'ACF_VulFu_Confirmed', 'ACF_VulFu_Sym', 'ACF_VulFu_Vul', 'ACF_VulFu_CXR', 'ACF_VulFu_Xpert', 'ACF_VulFu_Yield', 'ACF_VulFu_TP', 'ACF_VulFu_DS_Fl', 'ACF_VulFu_DR_Fl', 'ACF_VulFu_DR_Sl']


In [10]:
mss_stats

,Key,Scenario,Pop0,FollowUp,Duration,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,...,ACF_VulFu_Sym,ACF_VulFu_Vul,ACF_VulFu_CXR,ACF_VulFu_Xpert,ACF_VulFu_Yield,ACF_VulFu_TP,ACF_VulFu_DS_Fl,ACF_VulFu_DR_Fl,ACF_VulFu_DR_Sl,Coverage
0,14,Baseline,2.383551,0.000000,0.0,0.073606,0.011286,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
1,14,Intervention,2.383551,0.000000,0.0,0.066767,0.009700,4.412911,4.408596,0.364335,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
2,14,Intervention,2.383551,0.000000,0.5,0.066859,0.009718,4.412903,4.408596,0.364353,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
3,14,Intervention,2.383551,0.000000,1.0,0.066614,0.009661,4.412909,4.408596,0.364302,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
4,14,Intervention,2.383551,0.000000,1.5,0.066803,0.009706,4.412904,4.408596,0.364343,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,1498,Intervention,2.383192,0.166667,2.0,0.063852,0.007494,4.412673,4.407931,0.343466,...,5.116842,0.0,0.0,0.184982,0.013884,0.001188,0.001128,0.0,0.000060,0.2
6396,1498,Intervention,2.383192,0.166667,2.5,0.063716,0.007473,4.412708,4.407931,0.343443,...,5.749693,0.0,0.0,0.208002,0.015751,0.001486,0.001411,0.0,0.000074,0.2
6397,1498,Intervention,2.383192,0.166667,3.0,0.063608,0.007455,4.412737,4.407931,0.343423,...,6.259510,0.0,0.0,0.226519,0.017229,0.001701,0.001616,0.0,0.000085,0.2
6398,1498,Intervention,2.383192,0.166667,3.5,0.063522,0.007440,4.412763,4.407931,0.343406,...,6.677886,0.0,0.0,0.241676,0.018408,0.001843,0.001751,0.0,0.000092,0.2
